## Загрузка данных

In [3]:
import pandas as pd 
import numpy as np

fold = pd.read_csv("./Data/fold.csv")
unfold = pd.read_csv("./Data/unfold.csv")

profile = pd.read_csv("./Data/profile.csv")

delayexp = pd.read_csv("./Data/delayexp.csv")

delay_fig4 = pd.read_csv("./Data/delay_fig4.csv")
plotmat_fig4 = pd.read_csv("./Data/plotmat_fig4.csv")
plotmat2_fig4 = pd.read_csv("./Data/plotmat2_fig4.csv")

delay_fig5 = pd.read_csv("./Data/delay_fig5.csv")
plotmat_fig5 = pd.read_csv("./Data/plotmat_fig5.csv")
plotmat2_fig5 = pd.read_csv("./Data/plotmat2_fig5.csv")

## Биофизическая кабеля модель

Здесь мы проверяем теорию о профиле возмущения в пассивном волокне, вызванном потенциалом действия в соседнем волокне.

Функция рендеринга модели: 

In [78]:
class fCable_biophys: 
    def __init__(self):
        self.dx = 0.1 # mm шаг
        self.X = 110 # mm вся длина
        self.l = 0.002 # длина Рванье
        self.N = 200
        self.Vtrack = 40
        self.dt = 5e-5 # in ms
        self.T = 1e2 # in ms время моделирование
        # HH parameters: (2nd row: Brill et al. '77)
        self.gNa = 120 #проводимость канала Na
        #gNa = 1200; gNa = 4800; 
        self.eNa = 115 #обратный потенциал Na
        self.gK = 36 #проводимость канала K
        #gK = 90; gK = 720; 
        self.eK = -12 #обратный потенциал K
        self.gL = 0.3 #проводимость утечки
        #gL = 20; gL = 30; 
        self.g = 0.6 # общая проводимость
        
        #u - миелин
        #un - Ранвье
        self.tau = 0.47 # cable time constant ms
        self.taun = 0.03 # node time constant ms
        self.rm = 130 # продольное сопротивлление мейлина в M\Omega cm

    def render(self, dr, rho):
        dx =self.dx # mm шаг
        X = self.X # mm вся длина
        l = self.l # длина Рванье
        dt = self.dt # in ms
        T = self.T # in ms время моделирование
        # HH parameters: (2nd row: Brill et al. '77)
        #u - миелин
        #un - Ранвье
        tau = self.tau  # cable time constant ms
        taun = self.taun  # node time constant ms
        rm = self.rm # продольное сопротивлление мейлина в M\Omega cm
        
        x = self.dx * np.arange(1, np.ceil(X /dx) + 1)  # ceil - округление, вектор точек dx на кабеле
        r = 1 + dr * (1 / (self.N - 1)) * np.arange(self.N) # distribution of diameters
        # r = 1
        
        lambc = 1.93*r*(-np.log(self.g))**0.5 # cable length constant in mm
        lambn = 55e-3*(r)**0.5 # node length constant in mm
        sigrat = 1/3

        dn = np.round(0.2*r/dx) # растояние между перехватами 
        sn = 5 # расстояние от крайних узлов до края области моделирования
        nn = np.ceil((self.X-2*sn)/dx/dn) # количество узлов в каждом аксоне
        mask = np.zeros((self.N, len(x)), dtype=int) # массив шаблон, пустой пока

        dtype = [('num_indices', int), ('indices', object)]  
        idn = np.zeros(self.N, dtype=dtype)
        track = np.zeros(self.N, dtype=object)
        for o in range(self.N):
            #Заполняем маску
            start_index = int(sn / dx) 
            indices = start_index + np.arange(nn[o]) * dn[o] 
            indices = np.clip(indices, 0, mask.shape[1] - 1).astype(int)  
            mask[o, indices] = 1 

            mask[o, int(np.max(mask.shape) - sn / self.dx) - 1] = 1 

            idn[o]['num_indices'] = len(indices)
            idn[o]['indices'] = indices #сохраняем позиции dn в пространсве
            track[o] = np.zeros(len(indices)) #массив для записи параметров узлов для каждого нейрона

        trackn = np.zeros((self.N, 1))  
        trackt = np.zeros((self.N, 1)) 

        #временые и пространственные параметры
        lamb1 = lambc
        deltaL = l / dx
        lamb2 = ((1 - deltaL) * (1 / lambc**2) + deltaL * (1 / lambn**2))**(-1/2)
        tau1 = tau
        tau2 = lamb2**2 * ((1 - deltaL) * (tau / lambc**2) + deltaL * (taun / lambn**2))
        T = np.round(T/dt) #время моделирования 

        V = np.zeros((self.N, len(x)), dtype=float) #массив для записи V
        I = np.zeros((self.N, len(x)), dtype=float) #массив для записи I

        tau = tau1 * np.ones((self.N, len(x))) 
        lamb = np.outer(lamb1, np.ones(len(x)))

        #** переменные динамики модели Ходжкина-Хаксли:
        #np.max(np.sum(mask, axis=1)) - максимальное количесвто узлов в аксоне
        max_nodes = np.max(np.sum(mask, axis=1))
        Vn = np.zeros((self.N, max_nodes)) 
        m = 0.0529 * np.ones_like(Vn)
        h = 0.5961 * np.ones_like(Vn)
        n = 0.3177 * np.ones_like(Vn)
        Inode = np.zeros_like(Vn)

        #activation times
        ton = np.ones(self.N) / dt

        t = 0
        #пока массив trackt имеет незаполненые элементы и t<=T
        while (min(trackt)==0) and (t<=T): 
            t = t+1
    
            #Занимаемся I и V
            Istim = np.zeros(self.N)
            for o in range(self.N):
                Istim[o] = (1 / r[o]**2) * 1e4 * (t < 5e5 + ton[o]) * (t > ton[o])

            #заполняем массив напряжений в аксонах по всей видимости нулями на первом этапе
            for o in range(self.N):   
                num_nodes = int(nn[o] + 1)
                selected_elements = V[o, mask[o,:] == 1]
                num_to_assign = min(len(selected_elements), num_nodes)
                Vn[o, :num_to_assign] = selected_elements[:num_to_assign]
            #print(np.sum(V))
            
            #изменение управляющих параметров
            m = m + self.dt * (((2.5 - 0.1 * Vn) / (np.exp(2.5 - 0.1 * Vn) - 1)) * (1 - m) - (4 * np.exp(-Vn / 18)) * m)
            h = h + self.dt * (((0.07 * np.exp(-Vn / 20)) * (1 - h) - (1 / (np.exp(3.0 - 0.1 * Vn) + 1)) * h))
            n = n + self.dt * (((0.1 - 0.01 * Vn) / (np.exp(1 - 0.1 * Vn) - 1)) * (1 - n) - (0.125 * np.exp(-Vn / 80)) * n)
   
            r_repeated = np.tile(r[:, np.newaxis], (1, Inode.shape[1]))
            Inode = r_repeated * (self.gNa * m**3 * h * (self.eNa - Vn) + self.gK * n**4 * (self.eK - Vn)) / self.gL
            #(gNa * m**3 * h * (eNa - Vn) + gK * n**4 * (eK - Vn)) / gL - типичное уравнение тока ХХ модели 
            #r_repeated повторяет транспонированный вектор r по горизонтали, создавая матрицу с таким же количеством столбцов, как и у Inode
            #так мы получаем матрицу Inode с моделированными значениями
            Inode[:, 0] = Istim

            for o in range(self.N):       
                num_nodes = np.where(mask[o, :] == 1)  
                selected_elements = Inode[o, :int(nn[o] + 1)] 
                num_to_assign = min(len(selected_elements), len(num_nodes))
                I[o, mask[o, :] == 1][:num_to_assign] = selected_elements[:num_to_assign] #все столбцы где мы можем зафиксировать ток если элемент массива времени не заполнет и порог напряжения превышен (спайк возможен)
                #(trackt(o)==0)&&(V(o,1e3+sn/dx)>Vtrack)
                #print(f"ddd{V[o, 1000 + int(sn / self.dx)]} {self.Vtrack}")
                if (trackt[o] == 0) and (V[o, 1000 + int(sn / self.dx)] > self.Vtrack):
                    trackt[o] = t * self.dt

            Ve = np.concatenate((V[:, :1], V, V[:, -1:]), axis=1) #доп. матрица с небольши расширением
            Vxx = (Ve[:, 2:] + Ve[:, :-2] - 2 * Ve[:, 1:-1]) / dx**2 #видимо вторая производная V

            weights = (r**2 / np.sum(r**2)) / (1 + sigrat * ((1 - rho) / (self.g**2 * rho)))
            weighted_sum_Vxx = np.sum(weights[:, np.newaxis] * Vxx, axis=0)  
            weighted_sum_Vxx_broadcast = np.tile(weighted_sum_Vxx, (200,1)) 
            dV = (lamb**2 * Vxx -lamb**2 * weighted_sum_Vxx_broadcast - V + I) / tau

            V = V + dV

            #Занимаемся временой шкалой проверка можно ли выполнить моделирование за 100 шагов, наверное. 
            if t % 100 == 0:  
                for p in range(self.N):
                    #trackn - массив размером с кол-во аксонов (изначально пуст)
                    #nn - количество узлов в каждом аксоне (изначально не пуст)
                    if trackn[p] < nn[p]:  
                        #если хотябы в одной точке (что мы уже прошли) порог для спайка превышен. ? 
                        if np.sum(V[p, idn[p]['indices'][int(trackn[p] + 1):]] > self.Vtrack) > 0:
                            #находим эти точки и отбираем наиболее далекую. 
                            idt = np.argmax(V[p, indices_to_check] > self.Vtrack)
                    
                            trackn[p] += idt + 1 #обновляем шаг по аксонам
                            track[p][trackn[p]] = t * dt #вносит время преодоления порога в параметры, в нужный аксон

        out = {}  
        out['TM'] = np.mean(trackt)
        out['TD'] = np.std(trackt)
        out['delay'] = trackt
        out['track'] = track
        out['dn'] = dn.tolist() 

        return out

Само моделирование:

In [79]:
import time 

#dr = np.array([0.1, 0.2, 0.3])
#rho = np.array([0.5, 0.6, 0.7, 0.8, 0.9])

dr = [0.1]
#rho = np.arange(0.8, 0.9 + 0.01, 0.01)
#rho = np.array([0.8, 0.85, 0.87])
rho = np.arange(0.8, 0.95 + 0.025, 0.025) 

cc = np.zeros((len(dr), len(rho), 5), dtype=object)

for m, dr_val in enumerate(dr):
    for n, rho_val in enumerate(rho):
        biomodel = fCable_biophys()
        result = biomodel.render(dr_val, rho_val)
        for i, key in enumerate(result.keys()):
            cc[m, n, i] = result[key]

        timestamp = time.strftime('%H:%M:%S', time.localtime())
        print(f"Iteration ({m+1},{n+1})/{len(dr)},{len(rho)}, Timestamp: {timestamp}")

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ...

C:\Users\LKDat\AppData\Local\Temp\ipykernel_12340\1447498323.py:157: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  if np.sum(V[p, idn[p]['indices'][int(trackn[p] + 1):]] > self.Vtrack) > 0:


KeyboardInterrupt: 

In [40]:
cc[2, 3, 0] 

0.0